# Experiment Notebook: FEI Savings Rate Analyses

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to assess the impact that various settings of the FEI Savings Rate process have on User FEI Capital Allocation across Liquidity Pool, Money Market, and FEI Savings Deposits.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

from experiments.notebooks.helpers.system_metrics import *


In [ ]:
from experiments.notebooks.visualizations.plotly_theme import cadlabs_colorway_sequence

In [ ]:
from operator import lt, gt

In [ ]:
from scipy.stats import norm

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Configure Plotly
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
color_cycle = itertools.cycle(cadlabs_colorway_sequence)

In [ ]:
#line=dict(color=cadlabs_colorway_sequence[row - 1]),

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
cam_deposits = [
    "fei_liquidity_pool_user_deposit",
    "fei_money_market_user_deposit",
    "fei_savings_user_deposit",
    "fei_idle_user_deposit",
]

In [ ]:
parameter_overrides = {
#     "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
#     "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
#     "rebalancing_period": [int(365 / 4)],
#     "yield_withdrawal_period": [999],  # Disable yield-withdrawal policy
#     "yield_reinvest_period": [999],  # Disable yield-reinvestment policy
    "capital_allocation_fei_deposit_variables": [
            cam_deposits,
    ],
    "capital_allocation_rebalance_duration": [30],
    "fei_savings_rate_process": [
         lambda _run, timestep: 0.005,
         lambda _run, timestep: 0.03,
         lambda _run, timestep: 0.005 if timestep < 365 / 4 else (0.03 if timestep < 365 * 3/4 else 0.02),
         lambda _run, timestep: 0,
    ],
    #"money_market_utilization_rate_process": [
    #    lambda _run, timestep: 0.7, #+ gen_norm_rv(timestep, 0, 0.01),
    #]
}


In [ ]:
# Experiment configuration

# Override default experiment number of Monte Carlo Runs
simulation_1.runs = 100

# Override default experiment System Initial State
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

# Analysis Context

The goal of analysis notebooks is to be able to related the results of the FEI ecosystem model to the existing state of governance of the FEI protocol and ecosystem. To do this, we attempt to leverage the large scale simulation output capabilities of the radCAD framework to produce FIP-relevant qualitative and probabilistic conclusions, such that they may loosely aid in the conceptual phase of protocol decision support.

# Analysis 1: Effect of FEI Savings Rate on User FEI Capital Allocation

In this analysis we aim to explore FEI ecosystem model output in terms of movements of User FEI amongst idle and yield bearing deposits, and look at the sustainability of budgeting of various levels of the FEI savings rate in relation to protocol revenue.

In [ ]:
# NOTE: tba re budgeting

### Relevant FIPs for the Analysis

- FIP-73 (Contractionary Monetary Policy)
- FIP-103 (Creation of the FEI Savings Rate)
- FIP-105 (Volt)

In [ ]:
# TODO: give rationale

### Analysis Sections

- 0. Parameter sweeps for the analysis
- 1. CAM Weights and Deposit Allocation Proportions
    - FEI Savings Rate Setting 1
    - FEI Savings Rate Setting 2
    - FEI Savings Rate Setting 3
    - FEI Savings Rate Setting 4 (Disabled)
- 2. User Circulating FEI Deposit Evolution
    - 2.1. Total User Circulating FEI
    - 2.2. FEI Liquidity Pool Deposit
    - 2.3. FEI Money Market Deposit
    - 2.4. FEI Savings User Deposit
    - 2.5. FEI Idle User Deposit
- 3. State variables for mechanisms associated to FEI deposits
    - 3.1 FEI Money Market Dynamics
    - 3.2 FEI Liquidity Pool Dynamics
    - 3.3 FEI Savings Deposit Dynamics
- 4. Conclusion

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

### 0. Parameter Sweeps for the Analysis

In this analysis we look at the effect of setting the FEI savings rate in four ways:
- As a constant process below the money market supply rate
- As a constant process above the money market supply rate
- As a step function which moves between the two processes
- Disabled

We assume the demand for populating each deposit will depend on the rate of yield offered by the deposit, and gauge to what extent the difference in deposit yield affects how deposits are populated. We also look at the effect this has on Liquidity Pools, and a few other downstream effects on mechanism-specific metrics.

#### Visualization of parameter sweep:

We sweep the fei savings rate process for three values, two constant processes and a step function

In [ ]:
df.query('run==1').plot(x='timestamp', y=['fei_savings_rate'], color='subset')

A visualization of the volatile asset price trajectories used in each monte carlo run:

In [ ]:
df.query('subset==0').plot(x='timestamp', y=['volatile_asset_price'], color='run')

In [ ]:
use_cols = [key + '_balance' for key in cam_deposits]

### 1. CAM Weights and Deposit Allocation Proportions

#### 1.1 FEI Savings Rate Setting 1

In [ ]:
get_weight_evolution_average(df, 0, use_cols).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 1, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
#fig = px.area(df.query('subset==0'), x='timestamp', y=use_cols, groupnorm="percent")
fig = px.area(get_average_CAM_deposits(df, 0, use_cols), x='timestamp', y=use_cols, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 1",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
final_deposit_avgs = get_final_deposit_avgs(df, use_cols, 0)

fig = px.pie(final_deposit_avgs, values='value', names='deposit', hole=0.3)
fig.show()

#### 1.2 FEI Savings Rate Setting 2

In [ ]:
get_weight_evolution_average(df, 1, use_cols).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 2, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
#fig = px.area(df.query('subset==1'), x='timestamp', y=use_cols, groupnorm="percent")
fig = px.area(get_average_CAM_deposits(df, 1, use_cols), x='timestamp', y=use_cols, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 2",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
final_deposit_avgs = get_final_deposit_avgs(df, use_cols, 1)

fig = px.pie(final_deposit_avgs, values='value', names='deposit', hole=0.3)
fig.show()

#### 1.3 FEI Savings Rate Setting 3

In [ ]:
get_weight_evolution_average(df, 2, use_cols).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 3, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
#fig = px.area(df.query('subset==2'), x='timestamp', y=use_cols, groupnorm="percent")
fig = px.area(get_average_CAM_deposits(df, 2, use_cols), x='timestamp', y=use_cols, groupnorm="percent")


fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 3",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=0.99
    )
)

fig.show()

In [ ]:
final_deposit_avgs = get_final_deposit_avgs(df, use_cols, 2)

fig = px.pie(final_deposit_avgs, values='value', names='deposit', hole=0.3)
fig.show()

#### 1.4 FEI Savings Rate Setting 4

In [ ]:
get_weight_evolution_average(df, 3, use_cols).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 4, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
#fig = px.area(df.query('subset==2'), x='timestamp', y=use_cols, groupnorm="percent")
fig = px.area(get_average_CAM_deposits(df, 3, use_cols), x='timestamp', y=use_cols, groupnorm="percent")


fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 4",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=0.99
    )
)

fig.show()

In [ ]:
final_deposit_avgs = get_final_deposit_avgs(df, use_cols, 3)

fig = px.pie(final_deposit_avgs, values='value', names='deposit', hole=0.3)
fig.show()

### 2. User Circulating FEI Deposit Evolution

#### 2.1 Total User Circulating FEI

In [ ]:
fn_dict = get_fn_dict()

In [ ]:
fig = get_averages_by_subset(df, ['total_user_circulating_fei']).plot(
    #x='timestep',
    y='total_user_circulating_fei',
    color='subset'
)

fig.update_layout(
    title="Total User Circulating FEI",
    xaxis_title="Timestamp",
    yaxis_title="Total User Circulating FEI",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

What is the implicit impact of the introduction of a FEI savings deposit on overall User Circulating FEI?

With an extra yield-bearning avenue for users to hold FEI, mint/redemption dynamics as a result of liquidity pool and money market FEI and asset circulation will differ. As a result, so will the total user circulating FEI.

We can analyze this by looking at the empiricial probability of the total user circulating FEI hitting predetermined levels in cases where the FEI savings deposit is enabled vs disabled.

In [ ]:
md_df = get_variable_mean_difference(df, 'total_user_circulating_fei')
vd_prob = get_variable_difference_emp_prob(md_df)
print_emp_prob_message(vd_prob)

In [ ]:
s_df = get_state_variable_emp_dist(df, 'total_user_circulating_fei')
get_empirical_probability_for_deposit(s_df, 246_000_000, var_type='state_var')

In [ ]:
s_df.T.hist(bins=30)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'total_user_circulating_fei')

<b>Conclusion:</b> As we can see, in scenarios where the FEI savings rate is higher, total user circulating FEI is lower - attesting to the functionality of the FEI savings deposit as a demand sink for FEI. This is seen by the lower average final value attained, in combination with the resulting empirical probability of circulating FEI being higher in scenarios where a fei demand sink is present being very low (around 0.25).

### CAM Constituent Deposits:

What is the average impact of monetary policy settings of a fei savings rate in relation to other yield bearing and idle avenues for user circulating FEI?

We can analyze this by looking at the empiricial probability of the size of the deposit FEI hitting predetermined levels of overall proportion of user circulating FEI attained by the deposit.

#### 2.2 FEI Liquidity Pool Deposit

In [ ]:
fig = get_averages_by_subset(df, ['fei_liquidity_pool_user_deposit_balance']).plot(
    #x='timestamp',
    y='fei_liquidity_pool_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Liquidity Pool Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI LP Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
md_df = get_variable_mean_difference(df, 'fei_liquidity_pool_user_deposit_balance')
vd_prob = get_variable_difference_emp_prob(md_df)
print_emp_prob_message(vd_prob)

In [ ]:
p_df = get_deposit_proportion(df, 'fei_liquidity_pool_user_deposit_balance')
get_empirical_probability_for_deposit(p_df, 0.1)

In [ ]:
p_df.T.hist(bins=30)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_liquidity_pool_user_deposit_balance')

#### 2.3 FEI Money Market Deposit

In [ ]:
fig = get_averages_by_subset(df, ['fei_money_market_user_deposit_balance']).plot(
    #x='timestamp',
    y='fei_money_market_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Money Market Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI MM Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
md_df = get_variable_mean_difference(df, 'fei_money_market_user_deposit_balance')
vd_prob = get_variable_difference_emp_prob(md_df)
print_emp_prob_message(vd_prob)

In [ ]:
p_df = get_deposit_proportion(df, 'fei_money_market_user_deposit_balance')
get_empirical_probability_for_deposit(p_df, 0.3)

In [ ]:
p_df.T.hist(bins=30)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_user_deposit_balance')

#### 2.4 FEI Savings User Deposit

In [ ]:
fig = get_averages_by_subset(df, ['fei_savings_user_deposit_balance']).plot(
    #x='timestep',
    y='fei_savings_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Savings Deposit Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI SD Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
md_df = get_variable_mean_difference(df, 'fei_savings_user_deposit_balance')
vd_prob = get_variable_difference_emp_prob(md_df)
print_emp_prob_message(vd_prob)

In [ ]:
p_df = get_deposit_proportion(df, 'fei_savings_user_deposit_balance')
get_empirical_probability_for_deposit(p_df, 0.6)

In [ ]:
p_df.T.hist(bins=30)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_savings_user_deposit_balance')

#### 2.5 FEI Idle Deposit

In [ ]:
fig = get_averages_by_subset(df, ['fei_idle_user_deposit_balance']).plot(
    #x='timestep',
    y='fei_idle_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Savings Idle Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI Idle Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
md_df = get_variable_mean_difference(df, 'fei_idle_user_deposit_balance')
vd_prob = get_variable_difference_emp_prob(md_df)
print_emp_prob_message(vd_prob)

In [ ]:
p_df = get_deposit_proportion(df, 'fei_idle_user_deposit_balance')
get_empirical_probability_for_deposit(p_df, 0.1)

In [ ]:
p_df.T.hist(bins=30)

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_savings_user_deposit_balance')

### 3 State variables for Mechanisms associated to FEI deposits

#### 3.1 FEI Money Market Dynamics

In [ ]:
get_averages_by_subset(df, ['fei_money_market_borrowed']).plot(
    #x='timestamp',
    y=['fei_money_market_borrowed'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['fei_money_market_utilization']).plot(
    #x='timestamp',
    y=['fei_money_market_utilization'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['fei_money_market_supply_rate', 'fei_money_market_borrow_rate']).plot(
    #x='timestamp',
    y=['fei_money_market_supply_rate', 'fei_money_market_borrow_rate'],
    color='subset'
)

#### 3.2 FEI Liquidity Pool Dynamics

In [ ]:
get_averages_by_subset(df, ['liquidity_pool_trading_fees']).plot(
    #x='timestamp',
    y=['liquidity_pool_trading_fees'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['fei_liquidity_pool_user_deposit_yield_rate']).plot(
    #x='timestamp',
    y=['fei_liquidity_pool_user_deposit_yield_rate'],
    color='subset'
)

#### 3.1 FEI Savings Deposit Dynamics

In [ ]:
df.query('run==1').plot(x='timestamp', y=['fei_savings_rate'], color='subset')

## 4. Conclusion

From this analyisis we can make the following qualitative conclusion - when aggregate demand for deposits is based on yield rates as the only driver of utility, deposit population will converge to the mechanism offering the highest yield. We notice specifically that in trajectory 3, given the non-instantaneous rebalance velocity, that utilization of the FSD never reaches that attained in trajectory 2, but also does not fall back below that of trajectory 1.

We furthermore notice the existance of downstream effects on money market utilization and liquidity pool yield, more greatly sustained when both these deposits have higher affluence.

We finally notice some minor downstream effects on the collateralization ratio. Here, this process is affected by changes in PCV value and PCV yield accrued, as well as changes in user circulating FEI, the former affected by money market usage and the latter by liquidity pool usage.

# Appendix: Sustainability of FEI Savings Rate with protocol revenue

In the previous analysis we have inspected the effect of including a FEI savings rate on the circulation of user FEI driven by aggregate demand for yield.

Here, we are concerned with the sustainability of the FEI savings rate in relation to protocol revenue.
In practice, we attempt to answer how likely is it that protocol revenue can cover fei savings rate expenditure under different settings of the FEI savings  rate.

#### Analysis Setup

- Protocol revenue is the sum of PSM redemption fees and PCV yield
- FEI Savings Rate expenditure is the amount of FEI distributed to users of the FEI savings deposit, at the current size of the deposit and the current rate

We begin by looking at the average over all monte carlo runs of the evolution of FEI savings rate expenditure, PSM redemption fees, protocol revenue, and the KPI of interest, protocol profit.

In [ ]:
get_averages_by_subset(df, ['fsr_expenditure']).plot(
    #x='timestamp',
    y=['fsr_expenditure'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['psm_mint_redeem_fees']).plot(
    #x='timestamp',
    y=['psm_mint_redeem_fees'],
    color='subset'
)

In [ ]:
# NOTE percent of annualized yield to total pcv value should be in range of few percent

In [ ]:
get_averages_by_subset(df, ['protocol_revenue']).plot(
    #x='timestamp',
    y=['protocol_revenue'],
    color='subset'
)

### Protocol Profit

In [ ]:
get_averages_by_subset(df, ['protocol_profit']).plot(
    #x='timestamp',
    y=['protocol_profit'],
    color='subset'
)

In [ ]:
ts = '2022-10-01T12'
ts_df = get_first_negative_pr_timestep(df)
get_emp_prob_timestep(ts_df, ts)

Above, we compute the empirical probability that the profit is negative within 3 months of the start of the simulation. In the original sweep of savings rates, and conditional on the PCV yield and redemption fees being at their current value, the protocol can maintain each level of savings rate throughout the simulation.

### Synthetic Protocol Profit

We can look at 'synthetic' protocol profit and probability of budget deficit by setting higher levels of the FEI savings rate for realized values of protocol revenue.

In [ ]:
# Create a simulation for each analysis
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
parameter_overrides_2 = {
    "fei_savings_rate_process": [
         lambda _run, timestep: 0.1,
         lambda _run, timestep: 0.12,
         lambda _run, timestep: 0.15,
    ],
    "volatile_asset_price_process": [
        lambda _run, timestep: 2_000 + gen_norm_rv(timestep, 1, 1),
        lambda _run, timestep: 2_000 + gen_norm_rv(timestep, 1, 1),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365) + gen_norm_rv(timestep, 1, 1),
    ],
    # "yield_withdrawal_period": [int(365/4)],  # Toggle manually between policies in state update blocks
    #"yield_reinvest_period": [int(365/4)],
    #"money_market_utilization_rate_process": [
    #    lambda _run, timestep: 0.7, #+ gen_norm_rv(timestep, 0, 0.01),
    #]
}


In [ ]:
# Experiment configuration

# Override default experiment number of Monte Carlo Runs
simulation_2.runs = 1

# Override default experiment System Initial State
simulation_2.model.initial_state.update({})

# Override default experiment System Parameters
simulation_2.model.params.update(parameter_overrides_2)

In [ ]:
# Experiment execution
df2, exceptions2 = run(simulation_2)

In [ ]:
get_averages_by_subset(df2, ['pcv_yield']).plot(
    #x='timestamp',
    y=['pcv_yield'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df2, ['protocol_profit']).plot(
    #x='timestamp',
    y=['protocol_profit'],
    color='subset'
)

In [ ]:
ts = '2022-08-07T12'
ts_df = get_first_negative_pr_timestep(df2)
get_emp_prob_timestep(ts_df, ts)

In [ ]:
s_df = get_state_variable_emp_dist(df2, 'protocol_profit')
get_empirical_probability_for_deposit(-s_df, 50_000, var_type='state_var')

#### Conclusion

Protocol revenue and expenditure in practice are dependent on more complex dynamics that what the radCAD ecosystem model implements at its current level of abstraction. However, this simple experiment confirms the intuition that a constant FEI savings rate process in the order of 10-15% APY is not sustainable in the market conditions the simulation puts forth.